# Dependencies

In [ ]:
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam, RMSprop
from data.preprocessing import *
from data.data_utils import *
from models.macro_architectures import *
from models.df_models import *
from utils import *

# Defining variables

In [ ]:
sequence_length_hour = 5  #hour
sequence_length_minute = 300 #minute
pred_length = 5 #hours
dict_values = ['dst_kyoto', 'kp_gfz']

# Importing data and creating datasets

In [ ]:
#DATA PROCESSING
start_time = '20210101'
end_time = '20230802'
scrap_date = interval_time(start_time, end_time)
months = list(set([day[:6] for day in scrap_date]))
import_Dst(months)
l1_sample, l2_sample, dst, kp = automated_preprocessing(scrap_date, sep = True)
l1_sample_hour = (l1_sample[0].resample('60min').mean(), l1_sample[1].resample('60min').mean())
l2_sample_hour = (l2_sample[0].resample('60min').mean(), l2_sample[1].resample('60min').mean())

In [ ]:
hour_kp_dataset = KpData(l1_sample_hour, kp, 5, 3, hour=True, sep = True)
minute_kp_dataset = KpData(l1_sample_hour, kp, 5, 3, hour=False, sep = True)

In [ ]:
#Test:5% training: 95%

test_size = round(0.05*len(hour_kp_dataset))

train_hour_kp, test_hour_kp = random_split(hour_kp_dataset , [len(hour_kp_dataset) - test_size, test_size])

batch_size = 32  #Change based on GPU capacity

train_hour_kp_dl = DataLoader(train_hour_kp, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_hour_kp_dl = DeviceDataLoader(train_hour_kp_dl, device)
test_hour_kp_dl = DataLoader(test_hour_kp, batch_size*2, num_workers=4, pin_memory=True)
test_hour_kp_dl = DeviceDataLoader(test_hour_kp_dl, device)

# Hyperparameters

In [ ]:
##hyperparameters
epochs = 20
max_lr = 1e-2
weigth_decay = 1e-6
grad_clip = 1e-1
opt_func = Adam
#opt_func = RMSprop

# Models training